In [1]:
!pip install pandas
!pip install numpy
!pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from google.colab import drive

In [5]:
# version 1
def rename_columns_to_year_end(df):
    df_rt = df.copy()
    years = df_rt.columns.year
    year_end_dates = [pd.Timestamp(f"{year}-12-31") for year in years]
    df_rt.columns = year_end_dates
    return df_rt

def get_balance_sheet(ticker):
  balance_sheet_df = pd.DataFrame()
  n = len(ticker)
  print("start getting balance sheet")
  for c,i in enumerate(ticker):
    #retrive data from yfinance
    stock = yf.Ticker(i)
    raw = stock.balancesheet

    #rename column
    bs = rename_columns_to_year_end(raw)

    #reset index and rename it
    bs.reset_index(inplace=True)
    bs.rename(columns={'index': 'data'}, inplace=True)
    bs['symbol'] = i.split('.')[0]

    # Check for duplicates in the 'data' column and drop them
    bs = bs[~bs.duplicated(subset=['data'])]
    bs.set_index(['symbol', 'data'], inplace=True)

    #change type data of column for better look
    bs.columns = pd.to_datetime(bs.columns)

    #append to dataframe
    balance_sheet_df = pd.concat([balance_sheet_df, bs])
    print(f"{c+1}/{n} {i}")
  print("finish getting balance sheet")
  return balance_sheet_df

def get_income_statement(ticker):
  income_statement_df = pd.DataFrame()
  n = len(ticker)
  print("start getting income statement")
  for c,i in enumerate(ticker):
    #retrive data from yfinance
    stock = yf.Ticker(i)
    raw = stock.financials

    #rename column
    is_df = rename_columns_to_year_end(raw)

    #reset index and rename it
    is_df.reset_index(inplace=True)
    is_df.rename(columns={'index': 'data'}, inplace=True)
    is_df['symbol'] = i.split('.')[0]

    # Check for duplicates in the 'data' column and drop them
    is_df = is_df[~is_df.duplicated(subset=['data'])]
    is_df.set_index(['symbol', 'data'], inplace=True)

    #change type data of column for better look
    is_df.columns = pd.to_datetime(is_df.columns)

    #append to dataframe
    income_statement_df = pd.concat([income_statement_df, is_df])
    print(f"{c+1}/{n} {i}")
  print("finish getting income statement")
  return income_statement_df

def get_cashflow_statement(ticker):
  cashflow_statement_df = pd.DataFrame()
  n = len(ticker)
  print("start getting cashflow statement")
  for c,i in enumerate(ticker):
    #retrive data from yfinance
    stock = yf.Ticker(i)
    raw = stock.cashflow

    #rename column
    cf_df = rename_columns_to_year_end(raw)

    #reset index and rename it
    cf_df.reset_index(inplace=True)
    cf_df.rename(columns={'index': 'data'}, inplace=True)
    cf_df['symbol'] = i.split('.')[0]

    # Check for duplicates in the 'data' column and drop them
    cf_df = cf_df[~cf_df.duplicated(subset=['data'])]
    cf_df.set_index(['symbol', 'data'], inplace=True)

    #change type data of column for better look
    cf_df.columns = pd.to_datetime(cf_df.columns)

    #append to dataframe
    cashflow_statement_df = pd.concat([cashflow_statement_df, cf_df])
    print(f"{c+1}/{n} {i}")
  print("finish getting cashflow statement")
  return cashflow_statement_df

def get_monthly_stock_prices(ticker,start_date = "2022-01-01", end_date="2024-09-30"):
  stock = yf.Ticker(ticker)
  data = stock.history(start=start_date, end=end_date, interval="1mo")
  data = data.set_index(pd.to_datetime(data.index))

  # Resample to the start of each month
  monthly_prices = data['Close'].resample('MS').first()
  monthly_prices = pd.DataFrame(monthly_prices)
  monthly_prices['symbol'] = ticker.split('.')[0]
  monthly_prices.index = monthly_prices.index.to_period('M').to_timestamp(how='start')
  return monthly_prices

def get_dividend_data(ticker, start_date="2022-01-01", end_date="2024-09-30"):
  stock = yf.Ticker(ticker)
  dividends = stock.dividends
  # Filter dividends based on the specified date range
  dividends = dividends[(dividends.index >= start_date) & (dividends.index <= end_date)]
  if not dividends.empty:
    dividends_df = pd.DataFrame(dividends)
    dividends_df['symbol'] = ticker.split('.')[0]
    dividends_df = dividends_df.rename(columns={'Dividends':'amount'})
    dividends_df.index = dividends_df.index.to_period('M').to_timestamp(how='start')
    return dividends_df

def get_financial_data(ticker):
  balance_sheet = get_balance_sheet(ticker)
  income_statement = get_income_statement(ticker)
  cashflow_statement = get_cashflow_statement(ticker)
  return balance_sheet, income_statement, cashflow_statement

In [7]:
df = pd.read_csv('/content/set_scraped.csv')

In [8]:
bkk = df.copy()
bkk = bkk[['symbol', 'text-start', 'text-start 2', 'symbol 2', 'symbol 3']]
bkk.columns = ['symbol', 'company_name', 'market', 'industry', 'sector']
bkk['ticker'] = bkk['symbol'] + '.BK'
bkk.head(5)

,symbol,company_name,market,industry,sector,ticker
0,24CS,บริษัท ทเวนตี้ โฟร์ คอน แอนด์ ซัพพลาย จำกัด (ม...,mai,PROPCON,NaN,24CS.BK
1,2S,บริษัท 2 เอส เมทัล จำกัด (มหาชน),SET,INDUS,STEEL,2S.BK
2,3K-BAT,บริษัท ไทย เอ็นเนอร์จี สโตเรจ เทคโนโลยี จำกัด ...,SET,INDUS,AUTO,3K-BAT.BK
3,A,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน),SET,PROPCON,PROP,A.BK
4,A5,บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน),mai,PROPCON,NaN,A5.BK


In [9]:
set_list = bkk['ticker']
ticker_error = ['DIMET.BK']
set_list = set_list[set_list.isin(ticker_error) == False]

In [10]:
bl_st, ic_st, cf_st = get_financial_data(set_list)

start getting balance sheet
1/911 24CS.BK
2/911 2S.BK
3/911 3K-BAT.BK
4/911 A.BK
5/911 A5.BK
6/911 AAI.BK
7/911 AAV.BK
8/911 ABM.BK
9/911 ACAP.BK
10/911 ACC.BK
11/911 ACE.BK
12/911 ACG.BK
13/911 ADB.BK
14/911 ADD.BK
15/911 ADVANC.BK
16/911 ADVICE.BK
17/911 AE.BK
18/911 AEONTS.BK
19/911 AF.BK
20/911 AFC.BK
21/911 AGE.BK
22/911 AH.BK
23/911 AHC.BK
24/911 AI.BK
25/911 AIE.BK
26/911 AIMCG.BK
27/911 AIMIRT.BK
28/911 AIRA.BK
29/911 AIT.BK
30/911 AJ.BK
31/911 AJA.BK
32/911 AKP.BK
33/911 AKR.BK
34/911 AKS.BK
35/911 ALLA.BK
36/911 ALLY.BK
37/911 ALPHAX.BK
38/911 ALT.BK
39/911 ALUCON.BK
40/911 AMA.BK
41/911 AMANAH.BK
42/911 AMARC.BK
43/911 AMARIN.BK
44/911 AMATA.BK
45/911 AMATAR.BK
46/911 AMATAV.BK
47/911 AMC.BK
48/911 AMR.BK
49/911 ANAN.BK
50/911 ANI.BK
51/911 AOT.BK
52/911 AP.BK
53/911 APCO.BK
54/911 APCS.BK
55/911 APEX.BK
56/911 APO.BK
57/911 APP.BK
58/911 APURE.BK
59/911 AQUA.BK
60/911 ARIN.BK
61/911 ARIP.BK
62/911 ARROW.BK
63/911 AS.BK
64/911 ASAP.BK
65/911 ASEFA.BK
66/911 ASIA.BK
67/911 AS

In [11]:
bl_st

2019-12-31  \
symbol data                                                          
24CS   Treasury Shares Number                                  NaN   
       Ordinary Shares Number                                  NaN   
       Share Issued                                            NaN   
       Net Debt                                                NaN   
       Total Debt                                              NaN   
...                                                            ...   
ZIGA   Gross Accounts Receivable                               NaN   
       Cash Cash Equivalents And Short Term Investments        NaN   
       Cash And Cash Equivalents                               NaN   
       Cash Equivalents                                        NaN   
       Cash Financial                                          NaN   

                                                          2020-12-31  \
symbol data                                                            
24CS   Treasury Shares Number                                    NaN   
       Ordinary Shares Number                            430000000.0   
       Share Issued                                      430000000.0   
       Net Debt                                            9260000.0   
       Total Debt                                         24520000.0   
...                                                              ...   
ZIGA   Gross Accounts Receivable                                 NaN   
       Cash Cash Equivalents And Short Term Investments  197145469.0   
       Cash And Cash Equivalents                         197145469.0   
       Cash Equivalents                                  196905665.0   
       Cash Financial                                       239804.0   

                                                          2021-12-31  \
symbol data                                                            
24CS   Treasury Shares Number                                    NaN   
       Ordinary Shares Number                            430000000.0   
       Share Issued                                      430000000.0   
       Net Debt                                           42511083.0   
       Total Debt                                        106415812.0   
...                                                              ...   
ZIGA   Gross Accounts Receivable                         140935223.0   
       Cash Cash Equivalents And Short Term Investments  198968251.0   
       Cash And Cash Equivalents                         198968251.0   
       Cash Equivalents                                  197489583.0   
       Cash Financial                                      1478668.0   

                                                          2022-12-31  \
symbol data                                                            
24CS   Treasury Shares Number                                    NaN   
       Ordinary Shares Number                            430000000.0   
       Share Issued                                      430000000.0   
       Net Debt                                                  NaN   
       Total Debt                                         37403012.0   
...                                                              ...   
ZIGA   Gross Accounts Receivable                         151137224.0   
       Cash Cash Equivalents And Short Term Investments  106656345.0   
       Cash And Cash Equivalents                         106656345.0   
       Cash Equivalents                                  106123872.0   
       Cash Financial                                       532473.0   

                                                          2023-12-31  \
symbol data                                                            
24CS   Treasury Shares Number                                    0.0   
       Ordinary Shares Number                            430000000.0   
       Share Issued                           

In [12]:
ic_st

2019-12-31   2020-12-31  \
symbol data                                                             
24CS   Tax Effect Of Unusual Items                   NaN          0.0   
       Tax Rate For Calcs                            NaN     0.352007   
       Normalized EBITDA                             NaN   18310000.0   
       Total Unusual Items                           NaN          NaN   
       Total Unusual Items Excluding Goodwill        NaN          NaN   
...                                                  ...          ...   
ZIGA   Other Gand A                                  NaN   36088385.0   
       Gross Profit                                  NaN  200089296.0   
       Cost Of Revenue                               NaN  772842562.0   
       Total Revenue                                 NaN  972931858.0   
       Operating Revenue                             NaN  972931858.0   

                                                 2021-12-31    2022-12-31  \
symbol data                                                                 
24CS   Tax Effect Of Unusual Items                      0.0  547957.78384   
       Tax Rate For Calcs                          0.241495      0.223307   
       Normalized EBITDA                         35101165.0    41876591.0   
       Total Unusual Items                              0.0     2453829.0   
       Total Unusual Items Excluding Goodwill           0.0     2453829.0   
...                                                     ...           ...   
ZIGA   Other Gand A                              50404676.0    76244130.0   
       Gross Profit                             285825323.0    -1670702.0   
       Cost Of Revenue                          855457246.0  1147721781.0   
       Total Revenue                           1141282569.0  1146051079.0   
       Operating Revenue                       1141282569.0  1146051079.0   

                                                  2023-12-31 2024-12-31  
symbol data                                                              
24CS   Tax Effect Of Unusual Items             247811.882522        NaN  
       Tax Rate For Calcs                           0.195597        NaN  
       Normalized EBITDA                         -44136736.0        NaN  
       Total Unusual Items                         1266954.0        NaN  
       Total Unusual Items Excluding Goodwill      1266954.0        NaN  
...                                                      ...        ...  
ZIGA   Other Gand A                               52164613.0        NaN  
       Gross Profit                              123865854.0        NaN  
       Cost Of Revenue                           647043461.0        NaN  
       Total Revenue                             770909315.0        NaN  
       Operating Revenue                         770909315.0        NaN  

[42250 rows x 6 columns]

In [13]:
cf_st

2019-12-31   2020-12-31  \
symbol data                                                              
24CS   Free Cash Flow                                 NaN   -6460000.0   
       Repayment Of Debt                              NaN  -41160000.0   
       Issuance Of Debt                               NaN   38600000.0   
       Issuance Of Capital Stock                      NaN          0.0   
       Capital Expenditure                            NaN   -4010000.0   
...                                                   ...          ...   
ZIGA   Gain Loss On Investment Securities             NaN    7523384.0   
       Net Foreign Currency Exchange Gain Loss        NaN   -7273211.0   
       Gain Loss On Sale Of PPE                -1495326.0   -1061025.0   
       Gain Loss On Sale Of Business                  NaN          NaN   
       Net Income From Continuing Operations          NaN  117706970.0   

                                                 2021-12-31   2022-12-31  \
symbol data                                                                
24CS   Free Cash Flow                          -107938350.0 -200111753.0   
       Repayment Of Debt                       -106890926.0 -211047844.0   
       Issuance Of Debt                         183787534.0  130661611.0   
       Issuance Of Capital Stock                 93750000.0  473250000.0   
       Capital Expenditure                       -2596346.0   -5049928.0   
...                                                     ...          ...   
ZIGA   Gain Loss On Investment Securities               NaN     297060.0   
       Net Foreign Currency Exchange Gain Loss    -154329.0    -160516.0   
       Gain Loss On Sale Of PPE                         NaN          0.0   
       Gain Loss On Sale Of Business                    NaN          0.0   
       Net Income From Continuing Operations    204227961.0 -380891161.0   

                                                 2023-12-31 2024-12-31  
symbol data                                                             
24CS   Free Cash Flow                          -182411438.0        NaN  
       Repayment Of Debt                        -73975625.0        NaN  
       Issuance Of Debt                          64289339.0        NaN  
       Issuance Of Capital Stock                        0.0        NaN  
       Capital Expenditure                       -6998476.0        NaN  
...                                                     ...        ...  
ZIGA   Gain Loss On Investment Securities         -297060.0        NaN  
       Net Foreign Currency Exchange Gain Loss       6735.0        NaN  
       Gain Loss On Sale Of PPE                   1918168.0        NaN  
       Gain Loss On Sale Of Business             -5046074.0        NaN  
       Net Income From Continuing Operations     41567662.0        NaN  

[54201 rows x 6 columns]

In [14]:
bl_st.to_csv('balance_sheet.csv')
ic_st.to_csv('income_statement.csv')
cf_st.to_csv('cashflow_statement.csv')

from google.colab import files
files.download('balance_sheet.csv')
files.download('income_statement.csv')
files.download('cashflow_statement.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
#Unpivot balance
df_balance = pd.read_csv('balance_sheet.csv')
df_balance.drop('2019-12-31', axis = 1, inplace = True)

df_balance = df_balance.melt(
    id_vars=['symbol', 'data'],
    var_name='year',
    value_name='THB'
)

df_balance['type'] = 'balance'

df_balance.head()

,symbol,data,year,THB,type
0,24CS,Treasury Shares Number,2020-12-31,NaN,balance
1,24CS,Ordinary Shares Number,2020-12-31,430000000.0,balance
2,24CS,Share Issued,2020-12-31,430000000.0,balance
3,24CS,Net Debt,2020-12-31,9260000.0,balance
4,24CS,Total Debt,2020-12-31,24520000.0,balance


In [30]:
#Unpivot cashflow_statement
df_cashflow = pd.read_csv('cashflow_statement.csv')
df_cashflow.drop('2019-12-31', axis = 1, inplace = True)


df_cashflow = df_cashflow.melt(
    id_vars=['symbol', 'data'],
    var_name='year',
    value_name='THB'
)

df_cashflow['type'] = 'cashflow'

df_cashflow.head()

,symbol,data,year,THB,type
0,24CS,Free Cash Flow,2020-12-31,-6460000.0,cashflow
1,24CS,Repayment Of Debt,2020-12-31,-41160000.0,cashflow
2,24CS,Issuance Of Debt,2020-12-31,38600000.0,cashflow
3,24CS,Issuance Of Capital Stock,2020-12-31,0.0,cashflow
4,24CS,Capital Expenditure,2020-12-31,-4010000.0,cashflow


In [31]:
#Unpivot income
df_income = pd.read_csv('income_statement.csv')
df_income.drop('2019-12-31', axis = 1, inplace = True)


df_income = df_income.melt(
    id_vars=['symbol', 'data'],
    var_name='year',
    value_name='THB'
)

df_income['type'] = 'income'

df_income.head()

,symbol,data,year,THB,type
0,24CS,Tax Effect Of Unusual Items,2020-12-31,0.000000e+00,income
1,24CS,Tax Rate For Calcs,2020-12-31,3.520070e-01,income
2,24CS,Normalized EBITDA,2020-12-31,1.831000e+07,income
3,24CS,Total Unusual Items,2020-12-31,NaN,income
4,24CS,Total Unusual Items Excluding Goodwill,2020-12-31,NaN,income


In [35]:
#Union balance & cashflow & income
df = pd.concat([df_balance, df_cashflow, df_income])
df.head()

,symbol,data,year,THB,type
0,24CS,Treasury Shares Number,2020-12-31,NaN,cashflow
1,24CS,Ordinary Shares Number,2020-12-31,430000000.0,cashflow
2,24CS,Share Issued,2020-12-31,430000000.0,cashflow
3,24CS,Net Debt,2020-12-31,9260000.0,cashflow
4,24CS,Total Debt,2020-12-31,24520000.0,cashflow
